In [1]:
%display latex

In [2]:
reset()

## The Newman-Janis Algorithm
<p style='text-align: justify;'>The Newman-Janis Algorithm is a technique able to produce axisymmetric metrics from spherical ones, not necessarily on-shell.  The algorithm uses a coordinates complexification and a complex coordinate transformation to obtain a new axisymmetric metric from a seed metric.</p>


<p style='text-align: justify;'>After one can simplify the metric obtained by the Janis-Newman algorithm, often writing in Boyer-Lindquist coordinates, and verify if it satisfies equations of motion. Originally, using the Newman-Penrose tetrad formalism, the algorithm was the way to derivate the Kerr-Newman solution. The Janis-Newman algorithm has been successful to produce metrics.</p>

Starting from a spherically symmetric metric, our seed metric:

\begin{equation}
ds^2 = - A(r)dt^2 + B(r)^{-1}dr^2 + C(r)(d\theta^2 + \sin^2(\theta)d\phi^2).
\end{equation}

One can follow steps below:

* writing the seed metric in null coordinates, i.e., $\{t, r, \theta, \phi\} \mapsto \{u, r, \theta, \phi\}$;
* finding a Newman-Penrose frame $P^\mu_a = \{l^\mu, n^\mu, m^\mu, \bar{m}^\mu \} $ for inverse metric in null coordinates;
* performing the complex transformation:
\begin{equation}
r \mapsto  r+ia\cos(\theta), \;\;\;\;\; u \mapsto u-ia\cos(\theta)
\end{equation}
and a complexification of functions $A(r)$, $B(r)$ and $C(r)$;
* performing a change of coordinates;
* checking the equations of motion.

## Declaring the spacetime

Let's start by declaring the spacetime that its metric will be our seed. 

Our spacetime is a 4-dimensional Lorentzian manifold.

In [3]:
M = Manifold(4,'M',structure='Lorentzian')
X.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")
U.<u, r, th, ph> = M.chart(r"u r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")
K.<p, q, th, ph> = M.chart(r"p:u' q:(0,+oo):r' th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")

In [4]:
display(X.coord_range(),U.coord_range(), K.coord_range())

t: (-oo, +oo); r: (0, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

u: (-oo, +oo); r: (0, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

p: (-oo, +oo); q: (0, +oo); th: (0, pi); ph: [0, 2*pi] (periodic)

<p style='text-align: justify;'>The seed metric is a solution of Einstein field equations that describes the gravitational field outside a spherical mass. The line element $ds^2$ in Schwarzschild coordinates has the form</p>


\begin{equation}
ds^2 = - A(r)dt^2 + B(r)^{-1}dr^2 + C(r)(d\theta^2 + \sin^2(\theta)d\phi^2).
\end{equation}

In [5]:
#funções da métrica
#c = G = 1

A = function('A')(r)
B = function('B')(r)
C = function('C')(r)

In [6]:
#Inserimos aqui as entradas do tensor métrico - métrica "semente":
g = M.metric()
g[0,0] = -A
g[1,1] = 1/B
g[2,2] = C
g[3,3] = sin(th)**2*C

In [7]:
g.display()

g = -A(r) dt*dt + 1/B(r) dr*dr + C(r) dth*dth + C(r)*sin(th)^2 dph*dph

# Writing the seed metric in null coordinates

**1st step**: *writing the seed metric in outgoing Eddington-Finkelstein coordinates*. 

We perform the transformation:

\begin{equation}
 u= t -r^*.
\end{equation}

Introducing the tortoise coordinate $r^*$:

\begin{equation}
r^* = \int \frac{1}{\sqrt{A(r)B(r)}} dr.
\end{equation}

In [8]:
## Calculamos r* para a métrica semente, aqui r* = rt.
rt = integrate(1/sqrt(A*B),r); rt

integrate(1/sqrt(A(r)*B(r)), r)

We need to introduce a relation between our charts $X$ and $U$:

\begin{equation}
\left\{\begin{array}{ll} t = u + r^*\\ r = r\\ \theta = \theta \\ \phi = \phi \end{array}\right..
\end{equation}

Finally, we can write our line element in outgoing Eddington-Finkelstein coordinates.

In outgoing Eddington-Finkelstein coordinates, the seed metric can be written as:

\begin{equation}
ds^2 = - A(r)du^2 - 2\sqrt{\frac{A(r)}{B(r)}} dr^2 + C(r) (d\theta^2 + \sin^2(\theta)d\phi^2).
\end{equation}

In [9]:
#mudança de coordenadas de coordenadas de Schwarzschild (t,r,theta,phi) 
#para outgoing Eddington-Finkelstein (u,r,theta, phi)

X_to_U = U.transition_map(X, [u + rt, r, th, ph])

In [10]:
X_to_U.display()

t = u + integrate(1/sqrt(A(r)*B(r)), r)
r = r
th = th
ph = ph

In [11]:
X_to_U.inverse().display()

u = t - integrate(1/(sqrt(A(r))*sqrt(B(r))), r)
r = r
th = th
ph = ph

Using $g.display(U)$ we can verify if our seed metric has the form of outgoing Eddington-Finkelstein coordinates.

In [12]:
#elemento de linha no sistema de coordenadas (u, r, theta, phi)
g.display(U)

g = -A(r) du*du - sqrt(A(r))/sqrt(B(r)) du*dr - sqrt(A(r))/sqrt(B(r)) dr*du + C(r) dth*dth + C(r)*sin(th)^2 dph*dph

In [13]:
#elemento de linha no sistema de coordenadas (t, r, theta, phi)
g.display(X)

g = -A(r) dt*dt + 1/B(r) dr*dr + C(r) dth*dth + C(r)*sin(th)^2 dph*dph

In [14]:
#matriz tensor métrico sistema de coordenadas (t, r, theta, phi)
g[:]

[         -A(r)              0              0              0]
[             0         1/B(r)              0              0]
[             0              0           C(r)              0]
[             0              0              0 C(r)*sin(th)^2]

In [15]:
#matriz tensor métrico sistema de coordenadas (u, r, theta, phi)
g[U.frame(),:]

[                 -A(r) -sqrt(A(r))/sqrt(B(r))                      0                      0]
[-sqrt(A(r))/sqrt(B(r))                      0                      0                      0]
[                     0                      0                   C(r)                      0]
[                     0                      0                      0         C(r)*sin(th)^2]

## Finding a Newman-Penrose frame

**2nd step**: finding a Newman-Penrose frame.

For our seed metric in outgoing Eddington-Finkelstein coordinates, we can make the choice:

\begin{equation}
P^\mu_a = \{l^\mu, n^\mu, m^\mu, \bar{m}^\mu \},
\end{equation}
where:
\begin{equation}
\begin{split}
&l^\mu = \delta^\mu_r,\\
&n^\mu = \sqrt{\frac{B(r)}{A(r)}}\delta^\mu_u - \frac{B(r)}{2}\delta^\mu_r,\\
&m^\mu = \sqrt{\frac{1}{2C(r)}}\left(\delta^\mu_\theta + \frac{i}{\sin(\theta)}\delta^\mu_\phi\right)\\
&\bar{m}^\mu = \sqrt{\frac{1}{2C(r)}}\left(\delta^\mu_\theta - \frac{i}{\sin(\theta)}\delta^\mu_\phi\right).
\end{split}
\end{equation}

Any set of vector $P$ that satisfies:

\begin{equation}
\begin{split}
&g^{\mu \nu} = - l \otimes n - n \otimes l + m\otimes \bar{m} + \bar{m} \otimes m\\
&\langle l, l \rangle = \langle n, n \rangle = \langle m, m \rangle = \langle \bar{m}, \bar{m} \rangle = 0,\\
& \langle l, m \rangle = \langle l, \bar{m} \rangle = \langle n, m \rangle = \langle n, \bar{m} \rangle = 0,\\
& \langle l , n \rangle = -1,\\
& \langle m, \bar{m} \rangle = 1.
\end{split}
\end{equation}
is a Newman-Penrose frame.

Inner product is commutative, so switching the terms above do not change the relation. By the way, we assume the metric signature $[-,+,+,+]$. For this reason,

\begin{equation}
\langle m, \bar{m} \rangle = -\langle l , n \rangle = 1.
\end{equation}

If we assumed the signature $[+,-,-,-]$, 

\begin{equation}
\langle l , n \rangle = -\langle m, \bar{m} \rangle = 1.
\end{equation}

In [16]:
## Definimos o referencial de Newman-Penrose em relação às coordenadas de oEF.
# Importante dizer que definimos nossa tetrada no referencial U, o sage é capaz de determinar utilizando as cartas
#qual é a expressão para os vetores em X, U e K.
l = M.vector_field(name='l')
l[U.frame(), :, U] = [0, 1,0,0]

n = M.vector_field(name='n')
n[U.frame(), :, U] = [sqrt(B/A), -B/2,0,0]

m = M.vector_field(name='m')
m[U.frame(), :, U] = [0, 0, 1/sqrt(2*C),1/sqrt(2*C)*(I/sin(th))]

mb = M.vector_field(name='mb', latex_name=r'\bar{m}')
mb[U.frame(), :, U] = [0, 0, 1/sqrt(2*C),-1/sqrt(2*C)*(I/sin(th))]

e = [l,n,m,mb]

We should verify if our tetrad is a Newman-Penrose frame. We know that our choice satisfies the conditions. But if one made another choice, we should be able to verify it.

In [17]:
#Aqui verificamos se o produto interno dos vetores que compõem nosso referencial satisfazem as condições impostas
#para formarem um referencial de Newman-Penrose.
for i in range(4):
    for j in range(4):
        show(g(e[i],e[j]).display())

g(l,l): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0
   (p, q, th, ph) |--> 0

g(l,n): M --> R
   (t, r, th, ph) |--> -1
   (u, r, th, ph) |--> -1

g(l,m): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0
   (p, q, th, ph) |--> 0

g(l,mb): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0
   (p, q, th, ph) |--> 0

g(n,l): M --> R
   (t, r, th, ph) |--> -1
   (u, r, th, ph) |--> -1

...

g(m,mb): M --> R
   (t, r, th, ph) |--> 1
   (u, r, th, ph) |--> 1

g(mb,l): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0
   (p, q, th, ph) |--> 0

g(mb,n): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0
   (p, q, th, ph) |--> 0

g(mb,m): M --> R
   (t, r, th, ph) |--> 1
   (u, r, th, ph) |--> 1

g(mb,mb): M --> R
   (t, r, th, ph) |--> 0
   (u, r, th, ph) |--> 0

<p style='text-align: justify;'>If inner product satisfies the conditions, we should verify the relation between the metric and our seed metric, i.e., if we can write the seed metric as the tensorial product of $P^\mu_a$ vectors. </p>

To do this, we will define $P^\mu_a$ as a frame.

In [18]:
#Definimos o referencial P
P = M.vector_frame('P',e)

In [19]:
#referencial P no frame X. 
for j in range(4):
    show(P[j].display())

P_0 = 1/(sqrt(A(r))*sqrt(B(r))) d/dt + d/dr

P_1 = 1/2*sqrt(B(r))/sqrt(A(r)) d/dt - 1/2*B(r) d/dr

P_2 = 1/2*sqrt(2)/sqrt(C(r)) d/dth + 1/2*I*sqrt(2)/(sqrt(C(r))*sin(th)) d/dph

P_3 = 1/2*sqrt(2)/sqrt(C(r)) d/dth - 1/2*I*sqrt(2)/(sqrt(C(r))*sin(th)) d/dph

In [20]:
#nosso referencial no frame U.
for j in range(4):
    show(P[j].display(U))

P_0 = d/dr

P_1 = sqrt(B(r)/A(r)) d/du - 1/2*B(r) d/dr

P_2 = 1/2*sqrt(2)/sqrt(C(r)) d/dth + 1/2*I*sqrt(2)/(sqrt(C(r))*sin(th)) d/dph

P_3 = 1/2*sqrt(2)/sqrt(C(r)) d/dth - 1/2*I*sqrt(2)/(sqrt(C(r))*sin(th)) d/dph

In [21]:
#Aqui expressamos a métrica como o produto tensorial dos vetores.
g.display(P)

g = -P^0*P^1 - P^1*P^0 + P^2*P^3 + P^3*P^2

In [22]:
# Se ainda restar, dúvidas podemos determinar o produto g = -l*n - n*l + m*mb + mb*m.
# Devemos obter g^{\mu\nu}
(-l*n - n*l + m*mb + mb*m).display() #em X

-l*n-n*l+m*mb+mb*m = -1/A(r) d/dt*d/dt + B(r) d/dr*d/dr + 1/C(r) d/dth*d/dth + 1/(C(r)*sin(th)^2) d/dph*d/dph

In [23]:
(-l*n - n*l + m*mb + mb*m).display(U) #em U

-l*n-n*l+m*mb+mb*m = -sqrt(B(r))/sqrt(A(r)) d/du*d/dr - sqrt(B(r))/sqrt(A(r)) d/dr*d/du + B(r) d/dr*d/dr + 1/C(r) d/dth*d/dth + 1/(C(r)*sin(th)^2) d/dph*d/dph

In [24]:
#Para obter a métrica g_{\mu\nu}, basta descer os índices do produto.
((-l*n - n*l + m*mb + mb*m).down(g)).display() #em X

-A(r) dt*dt + 1/B(r) dr*dr + C(r) dth*dth + C(r)*sin(th)^2 dph*dph

In [25]:
((-l*n - n*l + m*mb + mb*m).down(g)).display(U) #em U 

-A(r) du*du - sqrt(A(r))/sqrt(B(r)) du*dr - sqrt(A(r))/sqrt(B(r)) dr*du + C(r) dth*dth + C(r)*sin(th)^2 dph*dph

## Complexification of $A(r)$, $B(r)$ and $C(r)$

**3rd step**: complexification of functions $A(r)$, $B(r)$ and $C(r)$;
<p style='text-align: justify;'>We perform a complexification of functions $A(r)$, $B(r)$ and $C(r)$. This procedure has some arbitrariness, but we can set some empirical rules. Such arbitrariness is the main reason for checking if the equations of motions are satisfied.</p>

Next, we present how these functions change under a complexification ([source](https://arxiv.org/abs/1701.00037)):

\begin{equation}
\frac{1}{r} \mapsto \frac{{\rm Re}r'}{|r'|^2},
\end{equation}

\begin{equation}
r \mapsto {\rm Re}r',
\end{equation}

\begin{equation}
r^2 \mapsto |r'|^2,
\end{equation}
com $r'\in \mathbb{C}$.

Any other complexification of a function could be expressed as combination of these presented, like $\frac{1}{r^2} \mapsto \frac{1}{|r'|^2}$.
 
*Example*:

In  outgoing Eddington-Finkelstein coordinates, Schwarzschild element line has the form:

\begin{equation}
-\left(1 - \frac{2m}{r}\right)du^2 - 2dudr + r^2d\theta^2 + r^2\sin(\theta)^2 d\phi^2.
\end{equation}

Therefore,

\begin{equation}
A(r) = B(r) = 1 - \frac{2m}{r}.
\end{equation}

Under a complexification, $A(r)$ transforms as:

\begin{equation}
1 - \frac{2m}{r} \mapsto 1 - \frac{2m {\rm Re}r'}{|r'|^2}.
\end{equation}

## Performing a complex transformation

**3rd step**: performing the complex transformation.

We perform a complex transformation from $U$ to $K$:

\begin{equation}
\left\{\begin{array}{ll} u = u' + ia\cos(\theta) \\ r = r'- ia\cos(\theta)\\ \theta = \theta \\ \phi = \phi \end{array}\right..
\end{equation}


In [26]:
a = var('a', domain='real')

In [27]:
#Definimos as complexificações das funções da métrica
ρ = function('ρ')(q, th, a) #ρ = |r'|
Ψ = function('Ψ')(q,th,a) #complexificação de B(r)
Φ = function('Φ')(q,th,a) #complexificação de A(r)
H = ρ**2 #complexificação de C(r)

In [28]:
#mudança de U para K
U_to_K = U.transition_map(K, [u - I*a*cos(th), r + I*a*cos(th), th, ph])

In [29]:
U_to_K.display()

p = -I*a*cos(th) + u
q = I*a*cos(th) + r
th = th
ph = ph

In [30]:
U_to_K.inverse().display()

u = I*a*cos(th) + p
r = -I*a*cos(th) + q
th = th
ph = ph

We need to input these complexification on our Newman-Penrose frame. Once we do this, we can no longer raising or lowering indices with our seed metric $g_{\mu\nu}$.

In [31]:
#lista (?) com os vetores de P.
complexificação = [e[z][U.frame(),:,U] for z in range(4)]; complexificação

[[0, 1, 0, 0],
 [sqrt(B(r)/A(r)), -1/2*B(r), 0, 0],
 [0, 0, 1/2*sqrt(2)/sqrt(C(r)), 1/2*I*sqrt(2)/(sqrt(C(r))*sin(th))],
 [0, 0, 1/2*sqrt(2)/sqrt(C(r)), -1/2*I*sqrt(2)/(sqrt(C(r))*sin(th))]]

In [32]:
#Substituremos os em cada um dos termos de cada um dos vetores as funções da métrica e guardaremos em outras listas.
#Cada lista referente a cada vetor de P.
lcomplexificado = []
for j in complexificação[0]:
    lcomplexificado.append(j.expr().subs({A:Φ, B: Ψ, C: H}))
    
ncomplexificado = []
for j in complexificação[1]:
    ncomplexificado.append(j.expr().subs({A:Φ, B: Ψ, C: H}))
    
mcomplexificado = []
for j in complexificação[2]:
    mcomplexificado.append(j.expr().subs({A:Φ, B: Ψ, C: H}))
    
mbcomplexificado = []
for j in complexificação[3]:
    mbcomplexificado.append(j.expr().subs({A:Φ, B: Ψ, C: H}))
    
#Redifinindo a tetrada após complexificação

l = M.vector_field(name='l')
l[U.frame(), :, U] = lcomplexificado

n = M.vector_field(name='n')
n[U.frame(), :, U] = ncomplexificado

m = M.vector_field(name='m')
m[U.frame(), :, U] = mcomplexificado

mb = M.vector_field(name='mb', latex_name=r'\bar{m}')
mb[U.frame(), :, U] = mbcomplexificado

ec = [l,n,m,mb]

In [33]:
for j in range(4):
        show(ec[j].display(U))

l = d/dr

n = sqrt(Ψ(q, th, a)/Φ(q, th, a)) d/du - 1/2*Ψ(q, th, a) d/dr

m = 1/2*sqrt(2)/sqrt(ρ(q, th, a)^2) d/dth + 1/2*I*sqrt(2)/(sqrt(ρ(q, th, a)^2)*sin(th)) d/dph

mb = 1/2*sqrt(2)/sqrt(ρ(q, th, a)^2) d/dth - 1/2*I*sqrt(2)/(sqrt(ρ(q, th, a)^2)*sin(th)) d/dph

In [34]:
#os vetores de P no frame K (após considerar a complexificação das funções).
for j in range(4):
    show(ec[j].display(K))

l = d/dq

n = sqrt(Ψ(q, th, a))/sqrt(Φ(q, th, a)) d/dp - 1/2*Ψ(q, th, a) d/dq

m = 1/2*I*sqrt(2)*a*sin(th)/abs(ρ(q, th, a)) d/dp - 1/2*I*sqrt(2)*a*sin(th)/abs(ρ(q, th, a)) d/dq + 1/2*sqrt(2)/abs(ρ(q, th, a)) d/dth + 1/2*I*sqrt(2)/(abs(ρ(q, th, a))*sin(th)) d/dph

mb = 1/2*I*sqrt(2)*a*sin(th)/abs(ρ(q, th, a)) d/dp - 1/2*I*sqrt(2)*a*sin(th)/abs(ρ(q, th, a)) d/dq + 1/2*sqrt(2)/abs(ρ(q, th, a)) d/dth - 1/2*I*sqrt(2)/(abs(ρ(q, th, a))*sin(th)) d/dph

After verifying $m$ e $\bar{m}$, we realize $\bar{m}$ do not keep the original relation with $m$. All $\bar{m}$'s componentes are the complex conjugates of $m$'s components. That relation must be maintained. The problem is our way to define $\bar{m}$, when Sage computes $\bar{m}$ after all transformations we did, it cannot keep this relation. The best way to correct it and keep the relation between vectors is rewriting $\bar{m}$ from $m$ after all transformations. We must redefine $\bar{m}$'s components as the complex conjugates of $m$'s componentes.

In [35]:
#Correção

mtermosconjugados = []

for j in ec[2][K.frame(),:,K]:
    mtermosconjugados.append(conjugate(j.expr()))
    
mb = M.vector_field(name='mb', latex_name=r'\bar{m}')
mb[K.frame(), :, K] = mtermosconjugados

Finally, we obtain the new metric computing the tensorial product:

\begin{equation}
- l \otimes n - n \otimes l + m\otimes \bar{m} + \bar{m} \otimes m.
\end{equation}

Our Newman-Penrose frame is not the same from starting point. After all transformations we did, $l^\mu$, $n^\mu$, $m^\mu$ and $\bar{m}^\mu$ were affected by them.



In [36]:
(-l*n - n*l + m*mb + mb*m).display(K)

-l*n-n*l+m*mb+mb*m = a^2*sin(th)^2/ρ(q, th, a)^2 d/dp*d/dp - (a^2*sin(th)^2*sqrt(Φ(q, th, a)) + sqrt(Ψ(q, th, a))*ρ(q, th, a)^2)/(sqrt(Φ(q, th, a))*ρ(q, th, a)^2) d/dp*d/dq + a/ρ(q, th, a)^2 d/dp*d/dph - (a^2*sin(th)^2*sqrt(Φ(q, th, a)) + sqrt(Ψ(q, th, a))*ρ(q, th, a)^2)/(sqrt(Φ(q, th, a))*ρ(q, th, a)^2) d/dq*d/dp + (a^2*sin(th)^2 + Ψ(q, th, a)*ρ(q, th, a)^2)/ρ(q, th, a)^2 d/dq*d/dq - a/ρ(q, th, a)^2 d/dq*d/dph + ρ(q, th, a)^(-2) d/dth*d/dth + a/ρ(q, th, a)^2 d/dph*d/dp - a/ρ(q, th, a)^2 d/dph*d/dq + 1/(sin(th)^2*ρ(q, th, a)^2) d/dph*d/dph

In [37]:
#g^{\mu\nu}
show((-l*n - n*l + m*mb + mb*m)[K.frame(),:, K])

[                                                                           a^2*sin(th)^2/ρ(q, th, a)^2 -(a^2*sin(th)^2*sqrt(Φ(q, th, a)) + sqrt(Ψ(q, th, a))*ρ(q, th, a)^2)/(sqrt(Φ(q, th, a))*ρ(q, th, a)^2)                                                                                                      0                                                                                        a/ρ(q, th, a)^2]
[-(a^2*sin(th)^2*sqrt(Φ(q, th, a)) + sqrt(Ψ(q, th, a))*ρ(q, th, a)^2)/(sqrt(Φ(q, th, a))*ρ(q, th, a)^2)                                              (a^2*sin(th)^2 + Ψ(q, th, a)*ρ(q, th, a)^2)/ρ(q, th, a)^2                                                                                                      0                                                                                       -a/ρ(q, th, a)^2]
[                                                                                                     0                                                                                                      0                                                                                       ρ(q, th, a)^(-2)                                                                                                      0]
[                                                                                       a/ρ(q, th, a)^2                                                                                       -a/ρ(q, th, a)^2                                                                                                      0                                                                            1/(sin(th)^2*ρ(q, th, a)^2)]

We can compute the inverse matrix of $(-l*n - n*l + m*mb + mb*m)[K.frame(),:, K]$ for writing the new line element.

In [38]:
#g_{\mu\nu}
ANJ = ((-l*n - n*l + m*mb + mb*m)[K.frame(),:, K]).inverse(); ANJ

[                                                                                                                   -Φ(q, th, a)                                                                                            -sqrt(Φ(q, th, a))/sqrt(Ψ(q, th, a))                                                                                                                               0                                   (a*sin(th)^2*Φ(q, th, a)*sqrt(Ψ(q, th, a)) - a*sin(th)^2*sqrt(Φ(q, th, a)))/sqrt(Ψ(q, th, a))]
[                                                                                           -sqrt(Φ(q, th, a))/sqrt(Ψ(q, th, a))                                                                                                                               0                                                                                                                               0                                                                                 a*sin(th)^2*sqrt(Φ(q, th, a))/sqrt(Ψ(q, th, a))]
[                                                                                                                              0                                                                                                                               0                                                                                                                   ρ(q, th, a)^2                                                                                                                               0]
[                                  (a*sin(th)^2*Φ(q, th, a)*sqrt(Ψ(q, th, a)) - a*sin(th)^2*sqrt(Φ(q, th, a)))/sqrt(Ψ(q, th, a))                                                                                 a*sin(th)^2*sqrt(Φ(q, th, a))/sqrt(Ψ(q, th, a))                                                                                                                               0 (2*a^2*sin(th)^4*sqrt(Φ(q, th, a)) - (a^2*sin(th)^4*Φ(q, th, a) - sin(th)^2*ρ(q, th, a)^2)*sqrt(Ψ(q, th, a)))/sqrt(Ψ(q, th, a))]

The new line element in advanced null coordinates has the form:

\begin{equation}
    ds'^2 = -\Phi du^2 - 2\sqrt{\frac{\Psi}{\Phi}}dudr - 2a\sin(\theta)^2\left(\sqrt{\frac{\Phi}{\Psi}} -\Phi\right)dud\phi  + 2a\sin(\theta)^2\sqrt{\frac{\Phi}{\Psi}}drd\phi  +\rho^2 d\theta^2 + \sin(\theta)^2\left\{\rho^2 + a^2\sin(\theta)^2\left(2\sqrt{\frac{\Phi}{\Psi}} - \Phi\right)\right\}d\phi^2.    
\end{equation}

We declare a new spacetime KS and input its metric as $ds'^2$ in Boyer-Lindquist coordinates.

Our line element $ds'^2$ in Boyer-Lindquist coordinates has the form 

\begin{equation}
ds'^2 = -\Phi dt^2  - 2a\sin(\theta)^2\left(\sqrt{\frac{\Phi}{\Psi}} -\Phi\right)dtd\phi + \frac{\rho^2}{ a^2\sin^2(\theta) - \rho^2\Phi } dr^2  +\rho^2 d\theta^2 + \sin(\theta)^2\left\{\rho^2 + a^2\sin(\theta)^2\left(2\sqrt{\frac{\Phi}{\Psi}} - \Phi\right)\right\}d\phi^2.    
\end{equation}

In [39]:
KS = Manifold(4,'KS',structure='Lorentzian')
S.<p, q, th, ph> = KS.chart(r"p:t q:(0,+oo):r th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")
dp, dq, dth, dph = S.coframe()[:]

In [40]:
gNJ = ANJ[0][0].expr()*dp*dp + (ANJ[0][3]+ANJ[3][0]).expr()*dp*dph + ANJ[2][2].expr()*dth*dth + ANJ[3][3].expr()*dph*dph + (ANJ[2][2].expr()/((ANJ[2][2]*ANJ[0][0]).expr() +a^2*sin(th)^2))*dq*dq; gNJ.display()

-Φ(q, th, a) dp*dp + 2*(a*sin(th)^2*Φ(q, th, a)*sqrt(Ψ(q, th, a)) - a*sin(th)^2*sqrt(Φ(q, th, a)))/sqrt(Ψ(q, th, a)) dp*dph + ρ(q, th, a)^2/(a^2*sin(th)^2 - Φ(q, th, a)*ρ(q, th, a)^2) dq*dq + ρ(q, th, a)^2 dth*dth + (2*a^2*sin(th)^4*sqrt(Φ(q, th, a)) - (a^2*sin(th)^4*Φ(q, th, a) - sin(th)^2*ρ(q, th, a)^2)*sqrt(Ψ(q, th, a)))/sqrt(Ψ(q, th, a)) dph*dph

In [41]:
gKS = KS.metric()
gKS= gNJ
gKS.display()

-Φ(q, th, a) dp*dp + 2*(a*sin(th)^2*Φ(q, th, a)*sqrt(Ψ(q, th, a)) - a*sin(th)^2*sqrt(Φ(q, th, a)))/sqrt(Ψ(q, th, a)) dp*dph + ρ(q, th, a)^2/(a^2*sin(th)^2 - Φ(q, th, a)*ρ(q, th, a)^2) dq*dq + ρ(q, th, a)^2 dth*dth + (2*a^2*sin(th)^4*sqrt(Φ(q, th, a)) - (a^2*sin(th)^4*Φ(q, th, a) - sin(th)^2*ρ(q, th, a)^2)*sqrt(Ψ(q, th, a)))/sqrt(Ψ(q, th, a)) dph*dph